In [1]:
import awswrangler as awr
import pandas as pd

file_path = r"C:\Users\raphael.almeida\Documents\Processos\historico_placas\sql\all_boards_sets.sql"
with open (file_path, 'r') as r:
    query = r.read()

df_sets = awr.athena.read_sql_query(query, database='silver')

criando cancelamentos e ativações

In [2]:
df_ativacoes = df_sets[df_sets['data_cancelamento_conjunto'].isnull()]

df_ativacoes = df_ativacoes[df_ativacoes['status_conjunto'].isin(['ATIVO', 'NOVO', 'RENOVAÇÃO'])] #averiguar o novo e renovação

df_ativacoes = df_ativacoes.drop_duplicates()

df_ativacoes['data_status'] = df_ativacoes['data_ativacao_conjunto']

df_ativacoes['vigencia'] = 'ATIVO'

In [3]:
df_cancelamentos = df_sets[df_sets['data_cancelamento_conjunto'].notna()]

df_cancelamentos = df_cancelamentos.drop_duplicates()

df_cancelamentos['data_status'] = df_cancelamentos['data_cancelamento_conjunto']

df_cancelamentos['vigencia'] = 'CANCELADO'

juntando as duas e filtrando ontem

In [4]:
df_aggregate = pd.concat([df_ativacoes, df_cancelamentos])

#declarando variáveis
today = pd.Timestamp.today()
today_date = today.date()
yesterday = today - pd.Timedelta(days=1)
yesterday_date= yesterday.date()
friday = today - pd.Timedelta(days=3)
friday_date = friday.date()
df_aggregate['data_status'] = pd.to_datetime(df_aggregate['data_status'])

if today.weekday() != 0:
    df_aggregate_yesterday = df_aggregate[df_aggregate['data_status'].dt.date==yesterday_date]
else:
    df_aggregate_yesterday = df_aggregate[df_aggregate['data_status'].dt.date==friday_date] 

#df_aggregate.sort_values('data_ativacao_conjunto', ascending=False)
df_aggregate_yesterday.sort_values('placa', ascending=False)

df_aggregate_yesterday['status'] = pd.NA
df_aggregate_yesterday['migration_from'] = pd.NA

df_aggregate_yesterday

C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_18664\4070733584.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aggregate_yesterday['status'] = pd.NA
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_18664\4070733584.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aggregate_yesterday['migration_from'] = pd.NA


,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao,data_status,vigencia,status,migration_from
1828,JQN8G17,9BD13561372028704,28126,12891,14433,2025-05-14,None,2025-05-19,ATIVO,Stcoop,2025-05-19,2025-05-19,ATIVO,<NA>,<NA>
5630,EJW9H06,9BSR6X400C3809015,1507,14733,14460,2025-05-16,None,2025-05-19,ATIVO,Stcoop,2025-05-19,2025-05-19,ATIVO,<NA>,<NA>
5753,AWA2A93,9BM958461AB728703,24658,22014,14456,2025-05-15,None,2025-05-19,ATIVO,Stcoop,2025-05-19,2025-05-19,ATIVO,<NA>,<NA>
5865,IPT7B66,9EP07133091002004,19087,22264,14472,2025-05-16,None,2025-05-19,ATIVO,Stcoop,2025-05-19,2025-05-19,ATIVO,<NA>,<NA>
5866,IPT7B77,9EP07123091002005,19086,22264,14472,2025-05-16,None,2025-05-19,ATIVO,Stcoop,2025-05-19,2025-05-19,ATIVO,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58468,EGK9J90,955L1523ABS316882,20444,7673,12722,2025-03-21,2025-05-19,2025-03-21,CANCELADO,Viavante,2025-05-19,2025-05-19,CANCELADO,<NA>,<NA>
58469,RZE2H52,93ZM2SSH0N8836864,16305291,7673,12722,2025-03-21,2025-05-19,2025-03-21,CANCELADO,Viavante,2025-05-19,2025-05-19,CANCELADO,<NA>,<NA>
58470,QYV6G87,93ZM2SSH0N8836128,16305283,7673,12722,2025-03-21,2025-05-19,2025-03-21,CANCELADO,Viavante,2025-05-19,2025-05-19,CANCELADO,<NA>,<NA>
58471,QYW1G59,98PTSH430MB117953,16305278,7673,12722,2025-03-21,2025-05-19,2025-03-21,CANCELADO,Viavante,2025-05-19,2025-05-19,CANCELADO,<NA>,<NA>


In [7]:
import openpyxl

file_path = r"C:\Users\raphael.almeida\Documents\Processos\historico_placas\historico_placas.xlsx"
df_base = pd.read_excel(file_path, 'historico_placas', engine='openpyxl')

In [8]:
df_base

,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao,data_status,vigencia,status,migration_from
0,RHM7G13,94BF1463MMV083531,226,350,377,2024-04-25,2025-05-16,2024-04-25,CANCELADO,Viavante,2025-05-16,2025-05-16,CANCELADO,CANCELADO,NaN
1,TAM3E07,9539J8TH8SR201806,187620,2423,3195,2024-09-17,2025-05-16,2024-09-16,CANCELADO,Viavante,2025-05-16,2025-05-16,CANCELADO,CANCELADO,NaN
2,BEB8J87,95388XZZ8LE001499,82191,350,377,2024-04-25,2025-05-16,2024-04-25,CANCELADO,Viavante,2025-05-16,2025-05-16,CANCELADO,CANCELADO,NaN
3,IUT0E60,9BVAG20C5DE796824,101841,407,446,2024-05-02,2025-05-16,2024-04-30,FINALIZADO,Viavante,2025-04-15,2025-05-16,CANCELADO,CANCELADO,NaN
4,HKH2J64,9BVAS02C0BE768289,15928766,5466,8322,2025-01-16,2025-05-16,2025-01-16,CANCELADO,Viavante,2025-05-16,2025-05-16,CANCELADO,CANCELADO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215076,ATE1464,9BVAS02CXAE758514,7094,16,23,2015-09-08,NaT,2015-09-11,RENOVAÇÃO,Segtruck,1754-08-30,2015-09-11,ATIVO,NaN,NaN
215077,HTS8130,9ADB06029AM299265,1536,16,23,2015-09-08,NaT,2015-09-11,RENOVAÇÃO,Segtruck,1754-08-30,2015-09-11,ATIVO,NaN,NaN
215078,DPC4574,9BVAS02C2AE759558,7095,16,23,2015-09-08,NaT,2015-09-11,RENOVAÇÃO,Segtruck,1754-08-30,2015-09-11,ATIVO,NaN,NaN
215079,AUK5813,955B0662BBS338181,1532,16,23,2015-09-08,NaT,2015-09-11,RENOVAÇÃO,Segtruck,1754-08-30,2015-09-11,ATIVO,NaN,NaN


criando as condições do status baeado no histórico montado

In [9]:
# Agrupar dados de conferência por chassi e benefício
df_base_grouped = df_base.groupby(['chassi']).agg({
    'empresa': list,
    'vigencia': list,
    'data_status': list
}).reset_index()

# Criar um dicionário para lookup rápido
conf_dict = df_base_grouped.set_index(['chassi']).to_dict('index')

# Processar cada chassi/benefício único (validação)
for chassi in df_aggregate_yesterday['chassi'].drop_duplicates().values:
    if chassi in conf_dict:
        conf_data = conf_dict[chassi]
        
        # Ordenar datas e status
        dates = sorted([d for d in conf_data['data_status'] if pd.notna(d)], reverse=True)
        
        if dates:  # Se existe pelo menos uma data
            ultima_data = dates[0]
            idx_ultima = conf_data['data_status'].index(ultima_data)
            vigencia_ultimo = conf_data['vigencia'][idx_ultima]
            empresa_ultima = conf_data['empresa'][idx_ultima]
            
            # Atualizar registros correspondentes
            mask = (df_aggregate_yesterday['chassi'] == chassi)
            empresa_atual = df_aggregate_yesterday.loc[mask, 'empresa'].iloc[0]
            vigencia_atual = df_aggregate_yesterday.loc[mask, 'vigencia'].iloc[0]
            
            if vigencia_atual == 'ATIVO':
                if vigencia_ultimo == 'CANCELADO':
                    df_aggregate_yesterday.loc[mask, 'status'] = 'REATIVAÇÃO'
                elif empresa_ultima != empresa_atual:
                    df_aggregate_yesterday.loc[mask, 'status'] = 'MIGRAÇÃO'
                    df_aggregate_yesterday.loc[mask, 'migration_from'] = empresa_ultima
                else:
                    df_aggregate_yesterday.loc[mask, 'status'] = 'RENOVAÇÃO'
            else:
                df_aggregate_yesterday.loc[mask, 'status'] = 'CANCELADO'
    else:
        mask = (df_aggregate_yesterday['chassi'] == chassi)
        df_aggregate_yesterday.loc[mask, 'status'] = 'NOVO'

df = pd.concat([df_aggregate_yesterday, df_base])
df.loc[df['vigencia'] == 'CANCELADO', 'status'] = 'CANCELADO'  #necessidade de forçar os com vigência CANCELADOS para status CANCELADO

df['data_registro'] = pd.to_datetime(df['data_registro'])
df['data_cancelamento_conjunto'] = pd.to_datetime(df['data_cancelamento_conjunto'])
df['data_ativacao_conjunto'] = pd.to_datetime(df['data_ativacao_conjunto'])
df['data_atualizacao'] = pd.to_datetime(df['data_atualizacao'])
df['data_status'] = pd.to_datetime(df['data_status'])

df['data_registro'] = df['data_registro'].dt.date
df['data_cancelamento_conjunto'] = df['data_cancelamento_conjunto'].dt.date
df['data_ativacao_conjunto'] = df['data_ativacao_conjunto'].dt.date
df['data_atualizacao'] = df['data_atualizacao'].dt.date
df['data_status'] = df['data_status'].dt.date
df['status'] = df['status'].fillna('NULL')
df['migration_from'] = df['migration_from'].fillna('NULL')

df = df.sort_values('data_status', ascending=False)
df['status'].value_counts()

status
NULL          110070
CANCELADO     104885
NOVO             222
REATIVAÇÃO       144
RENOVAÇÃO         36
MIGRAÇÃO          25
Name: count, dtype: int64

verificando se existem chassis que existam nos dois

In [11]:
# Verificando chassis existentes
chassis_yesterday = set(df_aggregate_yesterday['chassi'].unique())
chassis_all_active = set(df_base['chassi'].unique())

# Encontrando chassis que existem em ambos
chassis_comuns = chassis_yesterday.intersection(chassis_all_active)

# Encontrando chassis que só existem em df_aggregate_yesterday
chassis_apenas_yesterday = chassis_yesterday - chassis_all_active

print(f"Total de chassis únicos em df_aggregate_yesterday: {len(chassis_yesterday)}")
print(f"Total de chassis únicos em df_base: {len(chassis_all_active)}")
print(f"Total de chassis que existem em ambos: {len(chassis_comuns)}")
print(f"Total de chassis que só existem em df_aggregate_yesterday: {len(chassis_apenas_yesterday)}")

# Mostrando exemplos de chassis que só existem em df_aggregate_yesterday
if len(chassis_apenas_yesterday) > 0:
    print("\nExemplos de chassis que só existem em df_aggregate_yesterday:")
    for chassi in list(chassis_apenas_yesterday)[:5]:  # Mostrando os 5 primeiros
        print(f"Chassi: {chassi}")
        print(df_aggregate_yesterday[df_aggregate_yesterday['chassi'] == chassi][['placa', 'chassi', 'empresa', 'status_conjunto']].to_string())
        print()

Total de chassis únicos em df_aggregate_yesterday: 298
Total de chassis únicos em df_base: 93311
Total de chassis que existem em ambos: 212
Total de chassis que só existem em df_aggregate_yesterday: 86

Exemplos de chassis que só existem em df_aggregate_yesterday:
Chassi: 9BM951511SB401393
         placa             chassi   empresa status_conjunto
37781  SXM5G74  9BM951511SB401393  Viavante           ATIVO

Chassi: 9A9FB3154K9DS8817
         placa             chassi   empresa status_conjunto
57642  QKM5E01  9A9FB3154K9DS8817  Viavante           ATIVO

Chassi: 9BSG4X200C3807967
         placa             chassi   empresa status_conjunto
50062  AVW3D04  9BSG4X200C3807967  Viavante           ATIVO

Chassi: 98PTTH430RB142974
         placa             chassi   empresa status_conjunto
50080  SLW3G10  98PTTH430RB142974  Viavante           ATIVO

Chassi: 9ADG12430RM103903
         placa             chassi   empresa status_conjunto
50162  JYB3E31  9ADG12430RM103903  Viavante           ATIVO



para o excel

In [12]:
save_path = r'C:\Users\raphael.almeida\Documents\Processos\historico_placas\historico_placas.xlsx'
df.to_excel(save_path, engine = 'openpyxl', index = False, sheet_name= 'historico_placas')